# 采矿问题

## 目标和前提条件

在本示例中，您将学习如何建模和求解一个生产规划问题，该问题涉及优化一组矿山在五年期间的运营。

关于这种类型的模型的更多信息可以在 H. P. Williams 所著《数学规划中的模型构建》第五版的示例 #7（第 261-262 页和 310-312 页）中找到。

这个建模示例属于中级水平，我们假设您了解 Python 并熟悉 Gurobi Python API。此外，您还应该对构建数学优化模型有一定了解。

**下载代码库** <br />
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

---
## 问题描述

一家采矿公司需要为一个拥有四个矿山的区域制定五年运营计划。

在任何给定年份，他们在该区域最多只能运营三个矿山。但是，即使某个矿山在某一年可能不运营，如果预期将来会再次运营，公司仍然必须支付该矿山的特许权使用费。否则，可以永久关闭，无需再支付特许权使用费。

每个开放矿山（无论是否运营）的年度特许权使用费如下：

| <i></i> | 特许权使用费 |
| --- | --- |
| 矿山 1 | $\$5 百万$ |
| 矿山 2 | $\$4 百万$ |
| 矿山 3 | $\$4 百万$ |
| 矿山 4 | $\$5 百万$ |

每个矿山在一年内可以开采的矿石量有上限。这些限制如下：

| <i></i> | 最大产量 |
| --- | --- |
| 矿山 1 | $2.0\times10^6$ 吨 |
| 矿山 2 | $2.5\times10^6$ 吨 |
| 矿山 3 | $1.3\times10^6$ 吨 |
| 矿山 4 | $3.0\times10^6$ 吨 |

每个矿山生产的矿石品位不同。这个品位是按照一个标准来衡量的，当将矿石混合在一起时，其品质要求呈线性组合。例如，如果将两个不同矿山的相等数量的矿石混合在一起，得到的矿石品位将是这两种矿石品位的平均值。每个矿山的矿石品位如下：

| <i></i> | 矿石品位 |
| --- | --- |
| 矿山 1 | 1.0 |
| 矿山 2 | 0.7 |
| 矿山 3 | 1.5 |
| 矿山 4 | 0.5 |

每年，从每个运营矿山生产的矿石必须混合以生产特定品位的矿石。每年混合矿石的目标品位如下：

| <i></i> | 品位目标 |
| --- | --- |
| 第 1 年 | 0.9 |
| 第 2 年 | 0.8 |
| 第 3 年 | 1.2 |
| 第 4 年 | 0.6 |
| 第 5 年 | 1.0 |

最终混合的矿石售价为 $\$10$/吨。未来年份的收入和成本按照每年 $10\%$ 的比率折现。

采矿公司的关键问题是：每年应该运营哪些矿山，以及从每个矿山应该开采多少矿石？

这个问题是基于英国瓷土公司（English China Clays）面临的一个更大规模问题。在那个问题中（在 20 世纪 70 年代），目标是每年从 20 个矿山中选择最多 4 个进行运营。

---
## 模型构建

### 集合和索引

$t \in \text{Years}=\{1,2,\dots,5\}$: 年份集合。

$m \in \text{Mines}=\{1,2,\dots,4\}$: 矿山集合。

### 参数

$\text{price} \in \mathbb{R}^+$: 混合矿石每吨的售价（美元）。

$\text{max_mines} \in \mathbb{N}$: 任一年份可运营的最大矿山数量。

$\text{royalties}_m \in \mathbb{R}^+$: 保持矿山 $m$ 开放的年度特许权使用费（美元）。

$\text{capacity}_m \in \mathbb{R}^+$: 矿山 $m$ 每年可开采的最大矿石吨数。

$\text{quality}_m \in \mathbb{R}^+$: 矿山 $m$ 开采的矿石品位。

$\text{target} \in \mathbb{R}^+$: 第 $t$ 年混合矿石的品位目标。

$\text{time_discount}_t \in [0,1] \subset \mathbb{R}^+$: 第 $t$ 年收入和成本的时间折现系数。

### 决策变量

$\text{blend}_t \in \mathbb{R}^+$: 第 $t$ 年混合矿石的吨数。

$\text{extract}_{t,m} \in \mathbb{R}^+$: 第 $t$ 年从矿山 $m$ 开采的矿石吨数。

$\text{working}_{t,m} \in \{0,1\}$: 如果矿山 $m$ 在第 $t$ 年运营则为 1，否则为 0。

$\text{available}_{t,m} \in \{0,1\}$: 如果矿山 $m$ 在第 $t$ 年开放则为 1，否则为 0。

### 目标函数

- **利润**: 最大化规划期内的总利润（美元）。

\begin{equation}
\text{Maximize} \quad Z = \sum_{t \in \text{Years}}\sum_{m \in \text{Mines}}{\text{time_discount}_t*(\text{price}*\text{blend}_t-\text{royalties}_m*\text{available}_{t,m})}
\tag{0}
\end{equation}

### 约束条件

- **运营矿山**: 第 $t$ 年运营矿山的总数不能超过限制。

\begin{equation}
\sum_{m \in \text{Mines}}{\text{working}_{t,m}} \leq \text{max_mines} \quad \forall t \in \text{Years}
\tag{1}
\end{equation}

- **品位**: 第 $t$ 年混合矿石的最终品位必须满足目标。

\begin{equation}
\sum_{m \in \text{Mines}}{\text{quality}_m*\text{extract}_{t,m}} = \text{target}_t*\text{blended}_t \quad \forall t \in \text{Years}
\tag{2}
\end{equation}

- **质量守恒**: 第 $t$ 年开采的矿石总吨数应等于该年混合矿石的吨数。

\begin{equation}
\sum_{m \in \text{Mines}}{\text{extract}_{t,m}} = \text{blend}_t \quad \forall t \in \text{Years}
\tag{3}
\end{equation}

- **矿山产能**: 第 $t$ 年从矿山 $m$ 开采的矿石总吨数不能超过该矿山的年度产能。

\begin{equation}
\sum_{m \in \text{Mines}}{\text{extract}_{t,m}} \leq \text{capacity}_m*\text{working}_{t,m} \quad \forall t \in \text{Years}
\tag{4}
\end{equation}

- **开放运营**: 矿山 $m$ 只能在该年开放的情况下在第 $t$ 年运营。

\begin{equation}
\text{working}_{t,m} \leq \text{available}_{t,m} \quad \forall (t,m) \in \text{Years} \times \text{Mines}
\tag{5}
\end{equation}

- **关闭**: 如果矿山 $m$ 在第 $t$ 年关闭，将来不能再重新开放。

\begin{equation}
\text{available}_{t+1,m} \leq \text{available}_{t,m} \quad \forall (t < 5,m) \in \text{Years} \times \text{Mines}
\tag{6}
\end{equation}

---
## Python 实现

我们导入 Gurobi Python 模块和其他 Python 库。

In [ ]:
# %pip install gurobipy

In [1]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.11 & Gurobi 11.0

## 输入数据
我们定义模型的所有输入数据。

In [2]:
# 参数

years = [1, 2, 3, 4, 5]
mines = [1, 2, 3, 4]

royalties = {1: 5e6, 2: 4e6, 3: 4e6, 4: 5e6}
capacity = {1: 2e6, 2: 2.5e6, 3: 1.3e6, 4: 3e6}
quality  = {1: 1.0, 2: 0.7, 3: 1.5, 4: 0.5}
target = {1: 0.9, 2: 0.8, 3: 1.2, 4: 0.6, 5: 1.0}
time_discount = {year: (1/(1+1/10.0)) ** (year-1) for year in years}

max_mines = 3
price = 10

**注意：** 第 $n$ 期未来金额的现值由公式给出：$\text{present_value} = \frac{1}{(1+\text{interest_rate})^n}*\text{future_value} $

## 模型部署
我们创建一个模型和变量。对于每年和每个矿山，我们有 (i) 一个捕获生产量的变量，以百万吨为单位，(ii) 一个决策变量，告诉我们矿山是否开放，以及 (iii) 一个决策变量，告诉我们矿山是否运营。

In [3]:
mining = gp.Model('Mining')

blend = mining.addVars(years, name="Blend")
extract = mining.addVars(years, mines, name="Extract")
working = mining.addVars(years, mines, vtype=GRB.BINARY, name="Working")
available = mining.addVars(years, mines, vtype=GRB.BINARY, name="Available")

Set parameter LicenseID to value 2601452


接下来，我们插入约束条件。

每年只能有三个矿山运营。

In [4]:
#1. 运营矿山

OperatingMines = mining.addConstrs((working.sum(year, '*') <= max_mines for year in years), "Operating_mines")

矿山开采的矿石品位乘以开采量必须等于混合矿石需要的品位乘以混合矿石的数量。
这确保了品质标准得到满足。

In [5]:
#2. 品质

Quality = mining.addConstrs((gp.quicksum(quality[mine]*extract[year, mine] for mine in mines)
                   == target[year]*blend[year] for year in years), "Quality")

以下约束确保每年混合矿石的吨数等于组成部分的总吨数。

In [6]:
#3. 质量守恒

MassConservation = mining.addConstrs((extract.sum(year, '*') == blend[year] for year in years), "Mass Conservation")

以下约束确保矿山开采量不超过开采限制，并且只有在矿山运营时才有产出。

In [7]:
#4. 矿山产能

MineCapacity = mining.addConstrs((extract[year, mine] <= capacity[mine]*working[year, mine] for year, mine in extract), "Capacity")

以下约束确保当矿山运营时，它也需要处于开放状态。

In [8]:
# 开放运营
OpenToOperate = mining.addConstrs((working[year, mine] <= available[year, mine] for year, mine in available), "Open to Operate")

以下约束强制矿山在首次关闭后的所有年份都保持关闭状态。如果矿山关闭，以后就不能重新开放：

In [9]:
# 关闭矿山
ShutdownMine = mining.addConstrs((available[year+1, mine] <= available[year, mine]
                   for year, mine in available if year < years[-1]), "Shut down")

总利润由销售混合矿石的收入减去应付的特许权使用费组成。这是要最大化的目标。可以写作：

In [10]:
#0. 目标函数
obj = gp.quicksum(price*time_discount[year]*blend[year] for year in years) \
- gp.quicksum(royalties[mine] * time_discount[year] * available[year, mine] for year, mine in available)
mining.setObjective(obj, GRB.MAXIMIZE)

接下来，优化过程开始，Gurobi 找到最优解。

In [11]:
mining.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 71 rows, 65 columns and 182 nonzeros
Model fingerprint: 0xbc1c2f7b
Variable types: 25 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [5e-01, 3e+06]
  Objective range  [7e+00, 5e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 13 rows and 13 columns
Presolve time: 0.00s
Presolved: 58 rows, 52 columns, 135 nonzeros
Variable types: 16 continuous, 36 integer (36 binary)

Root relaxation: objective 1.577309e+08, 40 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0

---
## 分析

最优解产生了 $\$146.8620$ 百万的利润，各矿山在每年的生产计划如下（数量以百万吨为单位）：

### 开采计划
该计划确定了规划期内每年（行）从每个矿山（列）开采的矿石百万吨数。例如，在第 2 年将从矿山 3 开采 130 万吨矿石。

In [12]:
rows = years.copy()
columns = mines.copy()
extraction = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, mine in extract.keys():
    if (abs(extract[year, mine].x) > 1e-6):
        extraction.loc[year, mine] = np.round(extract[year, mine].x / 1e6, 2)
extraction

,1,2,3,4
1,2.00,0.00,1.3,2.45
2,0.00,2.50,1.3,2.20
3,1.95,0.00,1.3,0.00
4,0.12,2.50,0.0,3.00
5,2.00,2.17,1.3,0.00


### 销售计划
该计划定义了规划期内每年要销售的混合矿石百万吨数。例如，我们计划在第 4 年销售 562 万吨混合矿石。

In [14]:
rows = years.copy()
sales = pd.DataFrame(columns=['Sales'], index=rows, data=0.0)

for year in blend.keys():
    if (abs(blend[year].x) > 1e-6):
        sales.loc[year, 'Sales'] = np.round(blend[year].x / 1e6, 2)
sales

,Sales
1,5.75
2,6.00
3,3.25
4,5.62
5,5.47


**注意：** 如果您想将解决方案写入文件，而不是打印到终端，可以使用 model.write() 命令。示例实现为：

`mining.write("mining-output.sol")`

---
## 参考文献

H. Paul Williams，《数学规划中的模型构建》，第五版。

版权所有 © 2020 Gurobi Optimization, LLC